# Chutes and Ladders with SARSA Methods
This notebook uses SARSA to solve the Chutes and Ladders modified game.

The game board is shown below.  Players start on Square 0 (outside the board) and move towards the goal space (State 100).  Landing at the bottom of ladder moves the player to the top, while landing at the top of a chute moves the player to the bottom square.  


<img src="https://drive.google.com/uc?export=view&id=16k2EflsluUXCPVWVgL-vyv5-IzOqWZvW" alt="Drawing" width="500"/>


At each turn, the player may choose one of four dice (Effron Dice)
- Blue: 3,3,3,3,3,3
- Black: 4,4,4,4,0,0
- Red: 6,6,2,2,2,2
- Green: 5,5,5,1,1,1

The **purpose** of this code is to determine which dice to select at each turn so as to minimize the number of steps it takes to reach the goal state.  

In [ ]:
# import statements
import random
import numpy as np
import matplotlib.pyplot as plt


## State Transition Code

In [ ]:
def nextState (state, roll):
    '''
    This function transitions from the current state and current dice roll to the next state.
    INPUTS:
        state is the current state you are in (0 to 100)
        roll is the number showing on the dice (1 to 6)
    RETURN VALUE:
    this function returns the next state integer
    '''
    # we create a dictionary for the ladders and chutes.  The key is the start state of the chute/ladder
    # and the value is the ending state.  
    ladders = {1:38,4:14,9:31,21:42,28:84,36:44,51:67,71:91,80:100}
    chutes = {16:6,48:26,49:11,56:53,62:19,64:60,87:24,93:73,95:75,98:78}
    
    next_state = state + roll
    if next_state > 100:
        next_state = 100
    # now check for ladders
    if next_state in ladders:
        next_state = ladders[next_state]
    # now check for chutes
    if next_state in chutes:
        next_state = chutes[next_state]
        
    return next_state
    
def roll (dice_color):
    '''
    This function randomly rolls one of the four effron dice.  
    INPUT:
    dice_color should be among "red","blue","black", or "green"
    OUTPUT:
    an integer randomly selected from one of the dice
    '''
    
    if dice_color == 0:                         # red dice 
        return random.choice([2,2,2,2,6,6])
    if dice_color == 1:                         # blue dice 
        return 3
    if dice_color == 2:                         # black dice 
        return random.choice([0,0,4,4,4,4])
    if dice_color == 3:                         # green dice 
        return random.choice([1,1,1,5,5,5])
    # for invalid input
    return None

In [ ]:
def e_greedy(s, policy, epsilon):
    '''
    Implements an e-greedy policy.
    With probability epsilon, it returns random action choice
    otherwise returns action choice specified by the policy

    s = current state
    policy = policy function (an array that is indexed by state)
    epsilon (0 to 1) a probability of picking exploratory random action
    '''
    r = np.random.random()
    if r > epsilon:
        return policy[s]
    else:
        return random.randint(0,3)

In [ ]:
def init(): 
    '''
    Init rewards and policy 
    '''
    Q = np.zeros((101, 4)) 
    P = np.random.choice(4, 101)
    return Q, P 

In [ ]:
def SARSA(Q, policy, alpha, epsilon): 
    '''
    Perform 1 trial of learning
    '''
    state = 0   # state 0 = outside the board
    action = e_greedy(state, policy, epsilon)   # choose a dice 
    total_reward = 0  # number of throws

    while (state < 100): 
        next_state = nextState(state, roll(action))
        next_action = e_greedy(state, policy, epsilon)
        total_reward += 1
        TDerror = 1 + Q[next_state, next_action] - Q[state, action]
        Q[state, action] = Q[state, action] + alpha * TDerror
        state = next_state
        action = next_action 

    return total_reward 

In [ ]:
def do_trials(Q, policy, n, alpha, epsilon): 
    '''
    Perform n trials of learning 
    '''
    min_reward = 100       # random init
    for i in range(n): 
        min_reward = min(min_reward, SARSA(Q, policy, alpha, epsilon))
    return min_reward

In [ ]:
def policy_improvement(Q):
  '''
  Update value function V and policy P based on Q values
  '''
  V = np.min(Q, axis=1)
  P = np.argmin(Q, axis=1)
  return V, P

In [ ]:
Q, P = init()
m = 10
n = 100
epsilon = 0.1
alpha = 0.1
 
total_reward = 0        
for i in range(m):
  print("\n*** Trial {0:d} ***".format(i))
  reward = do_trials(Q, P, n, alpha, epsilon)
  total_reward += reward
  V, P = policy_improvement(Q)
  print("Perf: ", reward)

print("Average reward: {}".format(total_reward/m))

print(Q)
print(V)
print(P)




*** Trial 0 ***
Perf:  14

*** Trial 1 ***
Perf:  11

*** Trial 2 ***
Perf:  8

*** Trial 3 ***
Perf:  8

*** Trial 4 ***
Perf:  8

*** Trial 5 ***
Perf:  11

*** Trial 6 ***
Perf:  8

*** Trial 7 ***
Perf:  10

*** Trial 8 ***
Perf:  10

*** Trial 9 ***
Perf:  7
Average reward: 9.5
[[11.37015066 13.08635658 11.38536081 13.18489972]
 [ 0.          0.          0.          0.        ]
 [14.68256611  2.20740002  3.73623241  6.34446233]
 [ 4.84263879 14.71911951  4.65291835  7.51952275]
 [ 0.          0.          0.          0.        ]
 [ 5.61841952  8.48038479  6.37381268 14.04068633]
 [13.66853268 14.49991633 12.96813906 12.35628925]
 [ 7.99948977  9.49705937  8.51548742 11.54328518]
 [12.67429945 13.70825101 10.45029641 13.66597296]
 [ 0.          0.          0.          0.        ]
 [13.86427308  7.87085752 13.37466968 13.36377008]
 [12.65632285 13.67793438 12.92003036 14.28030691]
 [13.8489752  13.53009666 10.97750208 12.55258999]
 [12.92447134 12.22907224 11.62510585 11.76566807]
 